In [94]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice

In [95]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'

In [96]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [97]:
def unigrams(line):
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks =  re.sub(r"-\n", " ", notags)
    nodigits =  re.sub(r"\d+", " ", nobreaks)
    noroman =  re.sub(r"\b[XVILMC]+\b", "", nodigits)
    lower =   noroman.lower()
    return  re.findall(r'\w+', lower)

In [98]:
def make_bigrams(unigrams):
    bigram_tuples = window(unigrams,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [99]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items'] 
    year_filtered = filter(lambda item: year in item['judgmentDate'], judgements)
    yield from map(lambda item: item['textContent'], year_filtered)

In [100]:
json_files = os.listdir(json_data_dir)
judgements_files= filter(lambda name: re.match(filesInYearPattern, name), json_files)
texts =  chain.from_iterable(map( judgement_texts, judgements_files))
unigrams_it1, unigrams_it2 =tee(map(unigrams,texts))
unigrams =  chain.from_iterable(unigrams_it1)
bigrams = chain.from_iterable(map(make_bigrams,unigrams_it2))
   

counted_unigrams= Counter(unigrams)
counted_bigrams= Counter(bigrams)

# 10 najpopularniejszych bigramów w 2018

In [101]:
counted_bigrams.most_common(10)

[('z dnia', 11869),
 ('art k', 7184),
 ('w dniu', 5208),
 ('na podstawie', 4699),
 ('k p', 4686),
 ('w sprawie', 4657),
 ('z art', 4453),
 ('w tym', 4196),
 ('art ust', 3575),
 ('zgodnie z', 3378)]

# 10 najpopularniejszych unigramów w 2018

In [102]:
counted_unigrams.most_common(10) 

[('w', 118896),
 ('z', 69578),
 ('na', 42894),
 ('i', 39428),
 ('do', 37881),
 ('nie', 30528),
 ('o', 29826),
 ('k', 28916),
 ('r', 27356),
 ('że', 22479)]